# Exercício de ETL com Python

In [1]:
import pprint

dados_atores = []

with open('./data/actors.csv', mode='r', encoding='latin-1') as arquivo:
    linhas = arquivo.readlines()
    
    cabecalho = [col.strip() for col in linhas[0].split(',')]

    for linha in linhas[1:]:
        # --- CORREÇÃO 2: Substituir o caso específico problemático ANTES de processar ---
        linha_corrigida = linha.replace('"Robert Downey, Jr."', 'Robert Downey Jr')

        valores = [val.strip() for val in linha_corrigida.split(',')]

        # Pular linhas que não têm o número correto de colunas após o split
        if len(valores) != len(cabecalho):
            continue

        dicionario_linha = dict(zip(cabecalho, valores))

        try:
            # --- CORREÇÃO 1: Usar o nome exato da chave, com 'M' maiúsculo ---
            dicionario_linha['Total Gross'] = float(dicionario_linha['Total Gross'])
            dicionario_linha['Number of Movies'] = int(dicionario_linha['Number of Movies']) # Chave corrigida aqui
            dicionario_linha['Average per Movie'] = float(dicionario_linha['Average per Movie'])
            dicionario_linha['Gross'] = float(dicionario_linha['Gross'])
        except (ValueError, KeyError) as e:
            print(f"Aviso: Linha ignorada devido a erro. Erro: {e}. Linha: {linha.strip()}")
            continue

        dados_atores.append(dicionario_linha)

# --- Verificação Final ---
print("--- Verificação da Estrutura de Dados (3 primeiros registros) ---")
if dados_atores:
    pprint.pprint(dados_atores[:3])
else:
    print("A lista de dados ainda está vazia. Verifique os avisos acima.")

print(f"\nTotal de atores processados com sucesso: {len(dados_atores)}")

# --- FASE 2: Transformação e Carga ---

# Preparação: Criar os 5 arquivos de saída vazios
for i in range(1, 6):
    with open(f'etapa-{i}.txt', 'w') as arquivo:
        pass # 'pass' aqui significa apenas para criar o arquivo e não fazer nada

# --- Etapa 1: Apresente o ator/atriz com maior número de filmes ---

# 1. Inicializar variáveis para guardar o resultado
ator_com_mais_filmes = None
maximo_de_filmes = 0

# 2. Iterar sobre a lista de dicionários
for ator_info in dados_atores:
    # 3. Verificar se o número de filmes do ator atual é o maior encontrado até agora
    if ator_info['Number of Movies'] > maximo_de_filmes:
        # 4. Se for, atualizar nossas variáveis com os novos dados
        maximo_de_filmes = ator_info['Number of Movies']
        ator_com_mais_filmes = ator_info['Actor']

# 5. Formatar a string de resultado
resultado_etapa1 = f"O ator/atriz com maior número de filmes é {ator_com_mais_filmes}, com um total de {maximo_de_filmes} filmes."

# 6. Salvar o resultado no arquivo etapa-1.txt
with open('etapa-1.txt', 'w', encoding='utf-8') as arquivo:
    arquivo.write(resultado_etapa1)

print("Etapa 1 concluída. Resultado salvo em etapa-1.txt.")

# --- Etapa 2: Apresente a média de receita de bilheteria bruta dos principais filmes (coluna Gross) ---

# 1. Inicializar a soma. Usamos 0.0 para garantir que o resultado seja float.
soma_gross = 0.0

# 2. Iterar sobre a lista usando um nome de variável diferente para o item do loop (ex: ator_info)
for ator_info in dados_atores:
    # 3. Somar o valor da coluna correta ('Gross')
    soma_gross += ator_info['Gross']

# 4. Calcular a média APÓS o loop
total_de_atores = len(dados_atores) # len() na lista original, que não foi sobrescrita
media_gross = soma_gross / total_de_atores if total_de_atores > 0 else 0

resultado_etapa2 = f"A média de receita bruta dos principais filmes é de ${media_gross:.2f} milhões."

# 5. Salvar no arquivo
with open('etapa-2.txt', 'w', encoding='utf-8') as arquivo:
    arquivo.write(resultado_etapa2)

print("Etapa 2 concluída. Resultado salvo em etapa-2.txt.")

# --- Etapa 3: Apresente o ator/atriz com a maior média de receita por filme ---

# 1. Inicializar variáveis para guardar o melhor resultado encontrado até agora.
#    Começamos com um valor "impossível" (0.0) para garantir que o primeiro ator já seja maior.
ator_maior_media = None
max_media_por_filme = 0.0

# 2. Iterar sobre a lista de dicionários, um ator de cada vez.
for ator_info in dados_atores:
    
    # 3. Verificar se a média de receita do ator ATUAL é maior que a maior média que já guardamos.
    if ator_info['Average per Movie'] > max_media_por_filme:
        
        # 4. Se for maior, este ator é o novo "campeão". Atualizamos nossas variáveis.
        max_media_por_filme = ator_info['Average per Movie']
        ator_maior_media = ator_info['Actor']

# 5. Formatar a string de resultado final, usando os valores que guardamos.
resultado_etapa3 = f"O ator/atriz com a maior média de receita por filme é {ator_maior_media}, com uma média de ${max_media_por_filme:.2f} por filme."

# 6. Salvar o resultado no arquivo etapa-3.txt
with open('etapa-3.txt', 'w', encoding='utf-8') as arquivo:
    arquivo.write(resultado_etapa3)

print("Etapa 3 concluída. Resultado salvo em etapa-3.txt.")

# --- Etapa 4: Contagem de aparições dos filmes e ordenação ---

# 1. Usar um dicionário para contar a frequência de cada filme
contagem_filmes = {}
for ator_info in dados_atores:
    filme = ator_info['#1 Movie']
    # O método .get(chave, 0) retorna o valor atual ou 0 se a chave não existir.
    # Isso evita ter que checar "if filme in contagem_filmes:"
    contagem_filmes[filme] = contagem_filmes.get(filme, 0) + 1

# 2. Converter o dicionário para uma lista de tuplas (ex: [('The Avengers', 8), ...])
#    e ordenar com múltiplos critérios.
#    A lambda retorna uma tupla: (-contagem, nome_do_filme).
#    O Python ordena por tuplas elemento por elemento.
#    O sinal de menos (-) na contagem inverte a ordem, resultando em decrescente.
lista_filmes_ordenada = sorted(contagem_filmes.items(), key=lambda item: (-item[1], item[0]))

# 3. Salvar o resultado no arquivo etapa-4.txt
with open('etapa-4.txt', 'w', encoding='utf-8') as arquivo:
    # Usamos enumerate para gerar a sequência (1, 2, 3...)
    for sequencia, (filme, contagem) in enumerate(lista_filmes_ordenada, 1):
        linha_formatada = f"{sequencia} - O filme {filme} aparece {contagem} vez(es) no dataset\n"
        arquivo.write(linha_formatada)

print("Etapa 4 concluída. Resultado salvo em etapa-4.txt.")


# --- Etapa 5: Apresente a lista dos atores ordenada pela receita bruta (Total Gross) ---

# 1. Ordenar a lista 'dados_atores' usando uma lambda function como chave.
#    'key=lambda ator: ator['Total Gross']' diz ao Python para olhar o valor desta chave para ordenar.
#    'reverse=True' garante a ordem decrescente.
dados_atores_ordenados = sorted(dados_atores, key=lambda ator: ator['Total Gross'], reverse=True)

# 2. Salvar o resultado no arquivo etapa-5.txt
with open('etapa-5.txt', 'w', encoding='utf-8') as arquivo:
    # 3. Iterar pela NOVA lista já ordenada
    for ator_info in dados_atores_ordenados:
        linha_formatada = f"{ator_info['Actor']} - {ator_info['Total Gross']}\n"
        arquivo.write(linha_formatada)

print("Etapa 5 concluída. Resultado salvo em etapa-5.txt.")

FileNotFoundError: [Errno 2] No such file or directory: './data/actors.csv'